In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import tyro
import numpy as np
import imageio
import cv2

from src.modules.motion_extractor import MotionExtractor
from src.live_portrait_wrapper import LivePortraitWrapper
from src.modules.vqvae import VQVae

In [15]:
m_extr = MotionExtractor()
m_extr.load_pretrained(init_path="pretrained_weights/liveportrait/base_models/motion_extractor.pth")
m_extr.to('cuda')

Load pretrained model from pretrained_weights/liveportrait/base_models/motion_extractor.pth, ret: _IncompatibleKeys(missing_keys=['downsample_layers.0.0.weight', 'downsample_layers.0.0.bias', 'downsample_layers.0.1.weight', 'downsample_layers.0.1.bias', 'downsample_layers.1.0.weight', 'downsample_layers.1.0.bias', 'downsample_layers.1.1.weight', 'downsample_layers.1.1.bias', 'downsample_layers.2.0.weight', 'downsample_layers.2.0.bias', 'downsample_layers.2.1.weight', 'downsample_layers.2.1.bias', 'downsample_layers.3.0.weight', 'downsample_layers.3.0.bias', 'downsample_layers.3.1.weight', 'downsample_layers.3.1.bias', 'stages.0.0.dwconv.weight', 'stages.0.0.dwconv.bias', 'stages.0.0.norm.weight', 'stages.0.0.norm.bias', 'stages.0.0.pwconv1.weight', 'stages.0.0.pwconv1.bias', 'stages.0.0.grn.gamma', 'stages.0.0.grn.beta', 'stages.0.0.pwconv2.weight', 'stages.0.0.pwconv2.bias', 'stages.0.1.dwconv.weight', 'stages.0.1.dwconv.bias', 'stages.0.1.norm.weight', 'stages.0.1.norm.bias', 'stages

MotionExtractor(
  (detector): ConvNeXtV2(
    (downsample_layers): ModuleList(
      (0): Sequential(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm()
      )
      (1): Sequential(
        (0): LayerNorm()
        (1): Conv2d(96, 192, kernel_size=(2, 2), stride=(2, 2))
      )
      (2): Sequential(
        (0): LayerNorm()
        (1): Conv2d(192, 384, kernel_size=(2, 2), stride=(2, 2))
      )
      (3): Sequential(
        (0): LayerNorm()
        (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
      )
    )
    (stages): ModuleList(
      (0): Sequential(
        (0): Block(
          (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm()
          (pwconv1): Linear(in_features=96, out_features=384, bias=True)
          (act): GELU(approximate='none')
          (grn): GRN()
          (pwconv2): Linear(in_features=384, out_features=96, bias=True)
          (drop_path): Iden

In [16]:
with torch.no_grad():
    kp_vid = torch.stack([m_extr(image)['kp'].squeeze(0) for image in video_tensor])

In [17]:
N, K = kp_vid.shape

In [18]:
vqvae = VQVae(
    nfeats=K,
    code_num=512,
    code_dim=512,
    output_emb_width=512,
    down_t=2,
    stride_t=2,
    width=512,
    depth=3,
    dilation_growth_rate=3,
    norm=None,
    activation="relu",
    codebook_logger=None,
).to('cuda')

In [19]:
from train_tokenizer import Dataset

In [20]:
ds = Dataset("dataset")

In [25]:
with torch.no_grad():
    kp_vid = []
    for frame in ds[0]:
        image = frame.to('cuda')
        kp_vid.append(m_extr(image)['kp'].squeeze(0))
    kp_vid = torch.stack(kp_vid)
